In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MASS_DATA={"C": 12, "N": 14, "O": 16, "H": 1, "F": 19, "Cl": 36, "Br": 80, "I": 127, "S": 32, "P": 31, "Si": 28}
class AtomData:
    def __init__(self,element,partial_charge):
        self.q=partial_charge
        self.name=element
        
class MultiMol2Loader:
    def __init__(self,mol2file_name=None,allow_duplicates=False):
        self.filename=mol2file_name
        self.molecules=dict()
        self.atoms=dict()
        self.allow_duplicates=allow_duplicates
        if self.allow_duplicates==True:
            self.allow_duplicates="first"
        if mol2file_name is not None:
            self.loadFile(self.filename,self.allow_duplicates)

    def loadFile(self,filename,duplicates=False):
        mfile=open(filename,"r")
        started=False
        flags=[False,False,False]
        mcount=0
        for l in mfile:
            l=l.strip()
            #if not len(l): continue
            if "MOLECULE" in l:
                started=True
                named=False
                for f in flags: f=False
                continue
            if not started: continue
            if started and (not named):
                if not len(l):
                    l="Unnamed"+str(mcount)
                    mcount+=1
                if l in self.molecules:
                    if duplicates==False:
                        print("Molecule",l,"has a duplicate entry. Skipping")
                        started=False
                        continue
                    elif duplicates=="first":
                        started=False
                        continue
                    else:
                        raise ValueError("Duplicate Filter option '"+str(duplicates)+"' is not known")
                named=True
                molname=l.strip()
                self.molecules[molname]=[]
                self.atoms[molname]=[]
                continue
            elif started and named:
                self.molecules[molname].append(l)
                if "@<TRIPOS>" in l:
                    if "ATOM" in l: flags[0]=True
                    else: flags[0]=False
                else:
                    if flags[0]:
                        elname=l.split()[5].split(".")[0]
                        if elname not in MASS_DATA:
                            print("Bad element '"+elname+"' encountered in following line")
                            print(l.split())
                            raise ValueError()
                        #elname=l[42:46].strip().split(".")[0]
                        try: charge=float(l[-8:].strip())
                        except:
                            charge=0.0
                        self.atoms[molname].append(AtomData(elname,charge))

    def __getitem__(self,k):
        return self.molecules[k],self.atoms[k]

    def clearData(self):
        self.molecules=dict()
        self.atoms=dict()
        self.filename=None

    def writeMolecule(self,k,to_file=None):
        if str(k) not in self.molecules: raise ValueError("Molecule with key "+str(k)+" was not loaded")
        if to_file is not None: to_file.write("@<TRIPOS>MOLECULE\n")
        else: print("@<TRIPOS>MOLECULE")

        if to_file is not None: to_file.write(str(k)+"\n")
        else: print(str(k))

        for l in self.molecules[str(k)]:
            if to_file: to_file.write(l+"\n")
            else: print(l)

    def netCharge(self,k):
        if str(k) not in self.atoms: raise ValueError("Molecule with key "+str(k)+" was not loaded with atoms")
        q=0
        for at in self.atoms[k]:
            q+=at.q
        return q
    def netMass(self,k):
        if str(k) not in self.atoms: raise ValueError("Molecule with key "+str(k)+" was not loaded with atoms")
        m=0
        for at in self.atoms[k]:
            m+=MASS_DATA[at.name]
        return m

    def keys(self,from_atoms=False):
        if not from_atoms: return self.molecules.keys()
        else: return self.atoms.keys()

In [ ]:
print("Mol2 Reader Loaded")